## Diving into PostgreSQL: Practicing SQL queries with Python

In [1]:
import psycopg2
import pandas as pd

#### List all databases in the PostgreSQL server

In [3]:
# Connect to the PostgreSQL server
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="xxx"
)

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute the SQL command to list all databases
cur.execute("SELECT datname FROM pg_database")

# Fetch all rows and print them
rows = cur.fetchall()
for row in rows:
    print(row[0])
    
# Close the cursor and the connection
cur.close()
conn.close()

postgres
template1
template0
playtika
retailprices


#### Create new database

In [2]:
# Connect to PostgreSQL server
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="xxx",
    port="5432",
    database="postgres",
    connect_timeout=3,
)

# Set autocommit mode
conn.autocommit = True

# Create a new database
cur = conn.cursor()
cur.execute("CREATE DATABASE retailprices")

# Close cursor and connection
cur.close()
conn.close()

#### List all tables in the database

In [6]:
# Connect to the database
conn = psycopg2.connect(
    host="localhost",
    database="retailprices",
    user="postgres",
    password="xxx"
)

# Open a cursor to perform database operations
cur = conn.cursor()

# Query to get all table names from the current schema
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'")

# Fetch all the rows in a list of tuples
tables = cur.fetchall()

# Print the table names
for table in tables:
    table_name = table[0]
    print(table_name)
    cur.execute(f"SELECT * FROM {table_name} LIMIT 2")
    rows = cur.fetchall()
    print(f"Table: {table_name}")
    for row in rows:
        print(row)

# Close the cursor and the connection
cur.close()
conn.close()

districts
Table: districts
('5.0', 'TEL AVIV', 'תל אביב')
('4.0', 'CENTRAL', 'המרכז')
subdistricts
Table: subdistricts
('51.0', 'TEL AVIV', 'תל אביב', '5.0')
('53.0', 'TEL AVIV - HOLON', 'תל אביב - חולון', '5.0')
cities
Table: cities
('5000.0', 'TEL AVIV - YAFO', 'תל אביב -יפו', '51.0')
('6200.0', 'BAT YAM', 'בת ים', '53.0')
chains
Table: chains
('10', 'דור אלון')
('11', 'אושר עד')
subchains
Table: subchains
('10', 'AM:PM', '10')
('11', 'אושר עד', '11')
subchaintype
Table: subchaintype
('1', 'נוחות', '10')
('2', 'HD', '11')
stores
Table: stores
('1001', "אבן גבירול (פינת ז'בוטינסקי)", '10', '5000.0')
('1002', 'אבן גבירול 14', '10', '5000.0')


#### Insert new tables to database

In [4]:
# Read the Excel file into a pandas dataframe
all_data = pd.read_csv('C:/Users/israe/Documents/subjects/thesis/data/other datasets/Store_data_git.csv')

district_data = all_data[['DistrictID','DistrictName','DistrictNameHebrew']].drop_duplicates().reset_index(drop=True)
sub_district_data = all_data[['SubDistrictID','SubDistrictName','SubDistrictNameHebrew','DistrictID']].drop_duplicates().reset_index(drop=True)
city_data = all_data[['CityID','CityName','CityNameHebrew','SubDistrictID']].drop_duplicates().reset_index(drop=True)

chain_data = all_data[['ChainID','ChainName']].drop_duplicates().reset_index(drop=True)
sub_chain_data = all_data[['SubChainID','SubChainName','ChainID']].drop_duplicates().reset_index(drop=True)
sub_chain_type = all_data[['SubChainTypeID','SubChainTypeName','SubChainID']].drop_duplicates().reset_index(drop=True)

store_data = all_data[['StoreID','StoreName','SubChainID','CityID']].drop_duplicates().reset_index(drop=True)

all_data[:2]

,ChainID,ChainName,SubChainID,SubChainName,SubChainTypeID,SubChainTypeName,StoreID,StoreName,CityName,DistrictName,...,MetropolitanAreaID,MetropolitanAreaName,MunicipalStatusID,MunicipalStatusName,YishuvTypeID,YishuvTypeName,RegionalCouncilID,RegionalCouncilName,Unnamed: 40,StatisticalAreaPopulation
0,10,דור אלון,10,AM:PM,1,נוחות,1001,אבן גבירול (פינת ז'בוטינסקי),TEL AVIV - YAFO,TEL AVIV,...,110.0,תל אביב - גלעין,0.0,עירייה,130.0,יישוב עירוני,NaN,NaN,NaN,2621.179605
1,10,דור אלון,10,AM:PM,1,נוחות,1002,אבן גבירול 14,TEL AVIV - YAFO,TEL AVIV,...,110.0,תל אביב - גלעין,0.0,עירייה,130.0,יישוב עירוני,NaN,NaN,NaN,1391.667926


In [5]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="retailprices",
    user="postgres",
    password="xxx"
)

# Insert the data from the dataframe into a table in the database
cur = conn.cursor()

cur.execute('CREATE TABLE districts (DistrictID varchar, DistrictName varchar, DistrictNameHebrew varchar)')
for row in district_data.itertuples(index=False):
    cur.execute('INSERT INTO districts (DistrictID, DistrictName, DistrictNameHebrew) VALUES (%s, %s, %s)', row)

cur.execute('CREATE TABLE subdistricts (SubDistrictID varchar, SubDistrictName varchar, SubDistrictNameHebrew varchar, DistrictID varchar)')
for row in sub_district_data.itertuples(index=False):
    cur.execute('INSERT INTO subdistricts (SubDistrictID, SubDistrictName, SubDistrictNameHebrew, DistrictID) VALUES (%s, %s, %s, %s)', row)

cur.execute('CREATE TABLE cities (CityID varchar, CityName varchar, CityNameHebrew varchar, SubDistrictID varchar)')
for row in city_data.itertuples(index=False):
    cur.execute('INSERT INTO cities (CityID, CityName, CityNameHebrew, SubDistrictID) VALUES (%s, %s, %s, %s)', row)


cur.execute('CREATE TABLE chains (ChainID varchar, ChainName varchar)')
for row in chain_data.itertuples(index=False):
    cur.execute('INSERT INTO chains (ChainID, ChainName) VALUES (%s, %s)', row)
    
cur.execute('CREATE TABLE subchains (SubChainID varchar, SubChainName varchar, ChainID varchar)')
for row in sub_chain_data.itertuples(index=False):
    cur.execute('INSERT INTO subchains (SubChainID, SubChainName, ChainID) VALUES (%s, %s, %s)', row)

cur.execute('CREATE TABLE subchaintype (SubChainTypeID varchar, SubChainTypeName varchar, SubChainID varchar)')
for row in sub_chain_type.itertuples(index=False):
    cur.execute('INSERT INTO subchaintype (SubChainTypeID, SubChainTypeName, SubChainID) VALUES (%s, %s, %s)', row)

cur.execute('CREATE TABLE stores (StoreID varchar, StoreName varchar, SubChainID varchar, CityID varchar)')
for row in store_data.itertuples(index=False):
    cur.execute('INSERT INTO stores (StoreID, StoreName, SubChainID, CityID) VALUES (%s, %s, %s, %s)', row)
    
conn.commit()

# Close the connection
cur.close()
conn.close()

#### SQL queries

In [7]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="retailprices",
    user="postgres",
    password="xxx"
)

cur = conn.cursor()

In [8]:
print('Select all records from the "districts" table:')
cur.execute('SELECT * FROM districts')
rows = cur.fetchall()
for row in rows:
    print(row)

Select all records from the "districts" table:
('5.0', 'TEL AVIV', 'תל אביב')
('4.0', 'CENTRAL', 'המרכז')
('6.0', 'SOUTH', 'הדרום')
('1.0', 'JERUSALEM', 'ירושלים')
('3.0', 'HAIFA', 'חיפה')
('2.0', 'NORTH', 'הצפון')
('7.0', 'JUDEA AND SAMARIA', 'אזור יהודה והשומרון')
('NaN', 'NaN', '0')


In [9]:
print('Delete NaN records from the "districts" table:')
cur.execute('DELETE FROM districts WHERE DistrictID = %s', ('NaN',))
conn.commit()
cur.execute('SELECT * FROM districts')
rows = cur.fetchall()
for row in rows:
    print(row)

Delete NaN records from the "districts" table:
('5.0', 'TEL AVIV', 'תל אביב')
('4.0', 'CENTRAL', 'המרכז')
('6.0', 'SOUTH', 'הדרום')
('1.0', 'JERUSALEM', 'ירושלים')
('3.0', 'HAIFA', 'חיפה')
('2.0', 'NORTH', 'הצפון')
('7.0', 'JUDEA AND SAMARIA', 'אזור יהודה והשומרון')


In [10]:
print('Select the names of all cities in the subdistrict of Tel Aviv:')
cur.execute('SELECT CityName FROM cities WHERE SubDistrictID = %s', ('51.0',))
rows = cur.fetchall()
for row in rows:
    print(row[0])

Select the names of all cities in the subdistrict of Tel Aviv:
TEL AVIV - YAFO
HERZLIYYA
RAMAT HASHARON
GELIL YAM


In [11]:
print('Update the name of a city in the "cities" table:')
cur.execute('UPDATE cities SET CityName = %s WHERE CityName = %s', ('GALIL YAM', 'GELIL YAM'))
conn.commit()
cur.execute('SELECT CityName FROM cities WHERE SubDistrictID = %s', ('51.0',))
rows = cur.fetchall()
for row in rows:
    print(row[0])

Update the name of a city in the "cities" table:
TEL AVIV - YAFO
HERZLIYYA
RAMAT HASHARON
GALIL YAM


In [12]:
print('Select all records from the "subchains" table where SubChainID starts with "1":')
cur.execute("SELECT SubChainName FROM subchains WHERE SubChainID LIKE '1%'")
rows = cur.fetchall()
for row in rows:
    print(row)

Select all records from the "subchains" table where SubChainID starts with "1":
('AM:PM',)
('אושר עד',)
('אלונית בקיבוץ ובמושב',)
('ויקטורי',)
('ויקטורי באינטרנט',)
('חצי חינם',)
('יוחננוף',)
('יינות ביתן בסיטי',)
('יינות ביתן',)
('יש בשכונה',)


In [13]:
print('Select the names of all stores in the "stores" table that belong to "Victory" ChainID and city ID of Tel Aviv:')
cur.execute('SELECT StoreName FROM stores WHERE SubChainID = %s AND CityID = %s', ('13', '5000.0'))
rows = cur.fetchall()
for row in rows:
    print(row[0])

Select the names of all stores in the "stores" table that belong to "Victory" ChainID and city ID of Tel Aviv:
אוניברסיטת ת"א
דיזנגוף דיזינגוף
תל אביב לינקולן
תל אביב פלורנטין
תחנה מרכזית
תל אביב הארבעה


In [14]:
print('Select the top 10 frequent cities by the number of stores in the "stores" table:')
cur.execute('''SELECT CityName, COUNT(*) AS num_stores 
               FROM stores 
               JOIN cities ON stores.CityID = cities.CityID 
               WHERE CityName NOT IN (%s, %s) 
               GROUP BY CityName 
               ORDER BY num_stores DESC 
               LIMIT 10''', ('0', 'NaN'))
rows = cur.fetchall()
for row in rows:
    print(row)

Select the top 10 frequent cities by the number of stores in the "stores" table:
('TEL AVIV - YAFO', 173)
('JERUSALEM', 106)
('NESHER', 72)
('HAIFA', 67)
("BE'ER SHEVA", 44)
('NETANYA', 42)
('RISHON LEZIYYON', 42)
('ASHDOD', 38)
('PETAH TIQWA', 37)
('BENE BERAQ', 32)


In [15]:
print('Select the top 5 cities with the highest number of stores in the North district:')
cur.execute('''SELECT CityName, COUNT(*) as num_stores 
               FROM stores 
               JOIN cities ON stores.CityID = cities.CityID 
               JOIN subdistricts ON cities.SubDistrictID = subdistricts.SubDistrictID 
               JOIN districts ON subdistricts.DistrictID = districts.DistrictID 
               WHERE DistrictName = %s
               GROUP BY CityName 
               ORDER BY num_stores DESC 
               LIMIT 5''', ('NORTH',))
rows = cur.fetchall()
for row in rows:
    print(row)

Select the top 5 cities with the highest number of stores in the North district:
('AFULA', 12)
("KARMI'EL", 11)
('TIBERIAS', 11)
('NAHARIYYA', 9)
('QIRYAT MOTZKIN', 8)


In [16]:
print('Present the subdistricts with the number of AM:PM stores in each subdistrict:')
cur.execute("""SELECT s.SubDistrictName, COUNT(*) AS num_stores
               FROM stores AS st
               JOIN cities AS c ON st.CityID = c.CityID
               JOIN subdistricts AS s ON c.SubDistrictID = s.SubDistrictID
               JOIN subchains AS sc ON st.SubChainID = sc.SubChainID
               JOIN chains AS ch ON sc.ChainID = ch.ChainID
               WHERE s.SubDistrictName NOT IN (%s) AND sc.SubChainName IN (%s)
               GROUP BY s.SubDistrictName
               ORDER BY num_stores DESC""", ('NaN','AM:PM'))
rows = cur.fetchall()
for row in rows:
    print(row)

Present the subdistricts with the number of AM:PM stores in each subdistrict:
('TEL AVIV', 36)
('TEL AVIV - RAMAT GAN', 6)
('HADERA', 2)
('PETAH TIKVA', 2)
('TEL AVIV - HOLON', 1)
('REHOVOT', 1)


In [17]:
print('Present the subdistricts with the number of AM:PM stores in each subdistrict with at least 2 stores:')
cur.execute("""SELECT s.SubDistrictName, COUNT(*) AS num_stores
               FROM stores AS st
               JOIN cities AS c ON st.CityID = c.CityID
               JOIN subdistricts AS s ON c.SubDistrictID = s.SubDistrictID
               JOIN subchains AS sc ON st.SubChainID = sc.SubChainID
               JOIN chains AS ch ON sc.ChainID = ch.ChainID
               WHERE s.SubDistrictName NOT IN (%s) AND sc.SubChainName IN (%s)
               GROUP BY s.SubDistrictName
               HAVING COUNT(*) >= 2
               ORDER BY num_stores DESC""", ('NaN','AM:PM'))
rows = cur.fetchall()
for row in rows:
    print(row)

Present the subdistricts with the number of AM:PM stores in each subdistrict with at least 2 stores:
('TEL AVIV', 36)
('TEL AVIV - RAMAT GAN', 6)
('HADERA', 2)
('PETAH TIKVA', 2)


In [18]:
#######
#cur.execute("ROLLBACK")

In [19]:
# Close the connection
cur.close()
conn.close()

#### Delete tables from the database

In [20]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="retailprices",
    user="postgres",
    password="xxx"
)

cur = conn.cursor()

# Drop tables
cur.execute('DROP TABLE chains')
cur.execute('DROP TABLE subchains')
cur.execute('DROP TABLE subchaintype')
cur.execute('DROP TABLE stores')

# Commit the changes
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

#### Delete database from the server

In [21]:
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="xxx"
)

cur = conn.cursor()
cur.execute("ROLLBACK")
cur.execute("DROP DATABASE retailprices")

conn.commit()
conn.close()